In [1]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers
import time
import pylab
import matplotlib.pyplot as plt
import pickle
import csv

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
gpu_flag = 0

if gpu_flag >= 0:
    cuda.check_cuda_available()
xp = cuda.cupy if gpu_flag >= 0 else np


with open('/home/dl-box/Liver/sotsuron_train/train_pkl_data/new1024mix_NC_pseudocolor_classificate.pkl','rb') as f1:
    dataset1 = pickle.load(f1)
    
with open('/data1/20171127/Test_dataset/Cyst/NC/Cyst106_NC_pseudo_classificate.pkl','rb') as f2:
    dataset2 = pickle.load(f2)
    
train_set_x, train_set_y = dataset1[0]
test_set_x, test_set_y = dataset2[0]

train_set_x = np.array(train_set_x).astype(xp.float32)
train_set_y = np.array(train_set_y).astype(xp.int32)
test_set_x = np.array(test_set_x).astype(xp.float32)
test_set_y = np.array(test_set_y).astype(xp.int32)

x_train = train_set_x.reshape(train_set_x.shape[0],-1)
y_train = train_set_y

x_test = test_set_x.reshape(test_set_x.shape[0],-1)
y_test = test_set_y

N = x_train.shape[0]
N_test = y_test.size

x_train = x_train.reshape((len(x_train), 3, 29, 29))
x_test = x_test.reshape((len(x_test), 3, 29, 29))
x_train = (x_train*1.0)/255
x_test = (x_test*1.0)/255

class LeNet(chainer.Chain):
    def __init__(self):
        super(LeNet,self).__init__(
        conv1=F.Convolution2D(3, 96, 3),   
        conv2=F.Convolution2D(96, 192, 3), 
        conv3=F.Convolution2D(192, 192, 2),
        conv4=F.Convolution2D(192, 384, 2),
        conv5=F.Convolution2D(384, 384, 2),
        conv6=F.Convolution2D(384, 384, 2),
        conv7=F.Convolution2D(384, 768, 2),
        conv8=F.Convolution2D(768, 768, 2),
        conv9=F.Convolution2D(768, 768, 2),
        conv10=F.Convolution2D(768, 1536, 2),
        conv11=F.Convolution2D(1536, 1536, 2),
        conv12=F.Convolution2D(1536, 1536, 2),
            l1=F.Linear(6144, 128),            
            l2=F.Linear(128, 6)#classification
        )
        
    def __call__(self,x_data,train = True):
        #x = chainer.Variable(x_data)
        x = chainer.Variable(x_data.astype(np.float32))
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.relu(self.conv5(h))
        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        h = F.relu(self.conv8(h))
        h = F.relu(self.conv9(h))
        h = F.relu(self.conv10(h))
        h = F.relu(self.conv11(h))
        h = F.relu(self.conv12(h))
        h = F.dropout(F.relu(self.l1(h)), train= train)
        y = self.l2(h)
        y = F.relu(y)
        return y

    def forward_Train(self, x_data,y_data):
        y = self(x_data ,True)
        t = chainer.Variable(y_data)
        return F.softmax_cross_entropy(y, t), F.accuracy(y,t)
    
    def forward_Test(self, x_data, y_data):
        t = chainer.Variable(y_data)
        y = self(x_data)
        return F.accuracy(y, t)   
        
    def forward_label(self, x_data,y_data):
        label_list = np.empty((0,6),int)
        t = chainer.Variable(y_data)
        y = self(x_data)
        for i in range(0,y.data.shape[0]):
            classificate_max = 0
            classificate_max = max(y.data[i])
            for j in range(0,6):
                if classificate_max > y.data[i][j]:
                    y.data[i][j] = 0
                elif classificate_max == y.data[i][j]:
                    y.data[i][j] = 1     
                else:
                    y.data[i][j] = 0
            label_list = np.append(label_list, np.array([y.data[i]]), axis=0)
        label_list2 = sum(label_list)
        return label_list2

In [6]:
print len(train_set_x)
print len(train_set_y)
print len(test_set_x)
print len(test_set_y)


240000
240000
28627
28627


In [8]:
batchsize = 1000
n_epoch = 3

model = LeNet()
optimizer = optimizers.Adam()
optimizer.setup(model)

if gpu_flag >= 0:
    cuda.get_device(gpu_flag).use()
    model.to_gpu()
    
# ---Train---
for epoch in range(1, n_epoch + 1):
    print "epoch: %d" % epoch

    perm = np.random.permutation(N)
    for i in range(0, N, batchsize):
        x_batch = xp.asarray(x_train[perm[i:i + batchsize]],dtype=np.float32)
        y_batch = xp.asarray(y_train[perm[i:i + batchsize]],dtype=np.int32)
        
        model.zerograds()
        loss,train_acc = model.forward_Train(x_batch, y_batch)
        loss.backward()
        optimizer.update()

epoch: 1
epoch: 2
epoch: 3


In [9]:
#---Test(simple 6 classes clasificate)---

start_time = time.clock()
min_result = 1.000
acc_label2 = []
max_acc=0.0

    
sum_accuracy = 0   
for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])
    acc = model.forward_Test(x_batch, y_batch)
    sum_accuracy += float(acc.data) * len(y_batch)     
test_result = sum_accuracy / N_test

min_label = []
acc_label = np.empty((0,6),int)
for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])
    acc = np.array([model.forward_label(x_batch,y_batch)])
    acc_label = np.append(acc_label, np.array([acc[0]]), axis=0)
acc_label2 = sum(acc_label)/N_test

print("---probability---")
print "Liver : %f" %(acc_label2[0])
print "Cyst  : %f" %(acc_label2[1])
print "FNH   : %f" %(acc_label2[2])
print "CCC   : %f" %(acc_label2[3])
print "HCC   : %f" %(acc_label2[4])
print "Meta  : %f" %(acc_label2[5])
    
end_time = time.clock()
#print end_time - start_time

---probability---
Liver : 0.135699
Cyst  : 0.000000
FNH   : 0.170494
CCC   : 0.040710
HCC   : 0.012526
Meta  : 0.640571
